#### PROFITABLE APPS FOR THE APP STORE AND GOOGLE PLAY MARKET

The main aim of the project is to find the most profitable mobile app profiles. This projects is for companies which make free android and iOS mobile apps. The main source of revenue consist of in-app ads. The goal of the project is to analyze data to help our developers understand what type of apps are likely to attract more users. We will try to isolate the free apps from the dataset provided to us.

OPENING AND EXPLORING THE DATASET

[Android App Data](https://www.kaggle.com/lava18/google-play-store-apps) : Dataset containing approximately ten thousand android apps on Google playstore.

[iOS App Data](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps) : Dataset containing approximately seven thousand apps on Apple appstore.

Let's start by reading and exploring the datasets

In [2]:
opened_file=open('AppleStore.csv') #open dataset
from csv import reader #to pull data from csv file we csv module
read_file= reader(opened_file) #reader function to read the file
apps_data_iOS=list(opened_file) #transform file into list of lists
ios_header=apps_data_iOS[0]
ios=apps_data_iOS[1:]

In [3]:
opened_file=open('googleplaystore.csv')
from csv import reader
read_file= reader(opened_file)
apps_data_Android=list(read_file)
#print(apps_data_Android)
android_header=apps_data_Android[0]
android=apps_data_Android[1:]

Create explore dataset function such that it can be used again later. Function will help us to slice the dataset and count number of rows and columns in the dataset.

In [4]:
#dataset: List of lists
#start: starting index of the slice
#end: ending index of the slice
#rows_and_columns: If true, will give the count of number of rows and columns
def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]    
    for row in dataset_slice:
        print(row)
        print('\n') # adds a new (empty) line after each row

    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))

In [5]:
android_header = print(apps_data_Android[0]) #to get the column names of the data
explore_data(android,0,3,rows_and_columns=True)
explore_data(ios,0,3,rows_and_columns=True) #to slice the given dataset

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']
['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


Number of rows: 10841
Number of columns: 13
284882215,Facebook,389879808,USD,0.0,2974676,212,3.5,3.5,95.0,4+,Social Networking,37,1,29,1



389801252,Instagram,113954816,USD,0.0,2161558,1289,4.5,4.0,10.23,12+,Photo & Video,37,0,29,1



529479190,Clash of Clans,116476928,USD,0.0,2130805,

#### DATA CLEANING
Delete the incorrect row such that the data is uniform. Also, detect duplicates and remove them. The process of preparing data for analysis is called data cleaning.

In [6]:
del android[10472] #delete the row

In [7]:
android[10472] #check if the row has been deleted

['osmino Wi-Fi: free WiFi',
 'TOOLS',
 '4.2',
 '134203',
 '4.1M',
 '10,000,000+',
 'Free',
 '0',
 'Everyone',
 'Tools',
 'August 7, 2018',
 '6.06.14',
 '4.4 and up']

#### FIND DUPLICATE ROWS

In [8]:
for row in android:
    name = row[0]
    if name == 'Instagram':
        print(row)

['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577446', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66509917', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']


Some of the duplicate apps are Slack, Box, Instagram, Google Ads, Zenefits, HipChat - Chat Built for Teams etc. In order to remove the dupliucate entries we will remove the duplicate data such that we make the correct analysis.

In [9]:
#android[0:10]

#### COUNT OF DUPLICATES IN DATASET

To count the duplicates in the dataset we will create two empty lists.
If value is present in unique_list list we will append the value to duplicate_list else to the unique_list. In order to do so we will initialize both the lists and loop through the dataset.

In [10]:
duplicate_list = [] #create an empty list for duplicate names
unique_list = []    #create an empty list for unique names
for row in android:  
    name = row[0]
    if name in unique_list:
        duplicate_list.append(name)  #if name is unique, append to unique list else duplicate list
    else:
        unique_list.append(name)
print('Number of duplicate list ', len(duplicate_list)) #to find the length of the duplicate list
print('Number of unique list ', len(unique_list))       #to find the length of the unique list
print('Expected Length:', len(apps_data_Android[1:])-1181)  #to find the expected length post deleting the duplicates

Number of duplicate list  1181
Number of unique list  9659
Expected Length: 9660


We will not delete duplicates randomly. Keep the record with maximum reviews. That record with maximum views will be the latest record. Remove rest of the records.

In [36]:
reviews_max = {}
for row in android:
    name = row[0]       #name will be the key in the dictionary
    n_reviews = row[3]  #number of reviews will be the values in the dictionary
    if name in reviews_max and reviews_max[name] < n_reviews:  #check if the name is present and count is less than the one already available in the dictionary
        reviews_max[name]= n_reviews   #update the value
    elif name not in reviews_max:   #if not present, it will add name and count to the dictionary
        reviews_max[name]=n_reviews
#print(reviews_max)
len(reviews_max) #print length of the dictionary

9659

The count of the rows above should be expected count after removing the duplicates. Since there were 1181 duplicates, the count should be length of the dataset mius the duplicate values. Lets crosscheck

In [12]:
print('Expected length:', len(android)-1181) #to check if length is same as expected after removing duplication.
print('Actual length:', len(reviews_max))

Expected length: 9659
Actual length: 9659


In [13]:
android_clean = []
already_added = []

for app in android:
    name = app[0]
    n_reviews = app[3]
    
    if (reviews_max[name] == n_reviews) and (name not in already_added):
        android_clean.append(app)
        already_added.append(name) # make sure this is inside the if block

In [14]:
explore_data(android_clean, 0, 3, True)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


Number of rows: 9659
Number of columns: 13


#### REMOVING NON ENGLISH APPS

In the dataset we will notice that some of the data is not directed towards English speaking audience. Remove apps which are not directed towards English-speaking audience. According to ASCII range of english text lies between 0 to 127.

In [15]:
def name_english(string):
    for name in string:
        #print(name)
        if ord(name)>127:
            return False
    else:
        return True

In [16]:
print(name_english('Docs To Go™ Free Office Suite'))

False


But some English app names use emojis or other symbols that fall outside ASCII range. To minimize the data loss we will keep apps if the app name has less than 3 non-ASCII characters.

In [17]:
def name_english(string):
    non_english = 0
    for name in string:
        if ord(name)>127:
          non_english+=1
    if non_english>3:
        return False
    else:
        return True

In [18]:
name_english('Docs To Go™ Free Office Suite')

True

We will use name_english() function to filter out non-english apps from both the data-sets.

In [19]:
android_english_apps = []
ios_english_apps = []

for app in android_clean:
    name = app[0]
    if name_english(name):
        android_english_apps.append(app)
        
for app in ios:
    name = app[1]
    if name_english(name):
        ios_english_apps.append(app)
        
explore_data(android_english_apps, 0, 3, True)
print('\n')
explore_data(ios_english_apps, 0, 3, True)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


Number of rows: 9614
Number of columns: 13


284882215,Facebook,389879808,USD,0.0,2974676,212,3.5,3.5,95.0,4+,Social Networking,37,1,29,1



389801252,Instagram,113954816,USD,0.0,2161558,1289,4.5,4.0,10.23,12+,Photo & Video,37,0,29,1



529479190,Clash of Clans,116476928,USD,0.0,2130805,579,4.5,4.5,9.24.12,9+,Games,38,5,18,1



Number of rows: 7197
Number of columns: 93


#### ISOLATE FREE APPS

We will segregate the free apps for the analysis purpose since we need to analyse the revenue coming from all the free apps.

In [20]:
android_final = []
ios_final = []

for app in android_english_apps:
    price = app[7]
    if price == '0':
        android_final.append(app)
        
for app in ios_english_apps:
    price = app[4]
    if price == '0.0':
        ios_final.append(app)
        
print(len(android_final))
print(len(ios_final))

8862
0


We will left with -- android free apps and -- ios free apps.

#### MOST POPULAR APPS BY GENRE

To minimize risks and overhead, our validation strategy for an app idea is comprised of three steps:
1. Build a minimal Android version of the app, and add it to Google Play.
2. If the app has a good response from users, we develop it further.
3. If the app is profitable after six months, we build an iOS version of the app and add it to the App Store.

In [21]:
def freq_table(dataset, index):
    table = {}
    total = 0
    
    for row in dataset:
        total += 1
        value = row[index]
        if value in table:
            table[value] += 1
        else:
            table[value] = 1
    
    table_percentages = {}
    for key in table:
        percentage = (table[key] / total) * 100
        table_percentages[key] = percentage 
    
    return table_percentages

In [25]:
def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)
        
    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0])

We start by examining the frequency table for the category and genre column of the Google playstore data set.

In [22]:
print(freq_table(android_final, -4)) #frequency tables that show percentage
#print(android_header)

{'Puzzle;Creativity': 0.022568269013766643, 'Maps & Navigation': 1.399232678853532, 'Lifestyle;Pretend Play': 0.011284134506883321, 'Word': 0.2595350936583164, 'Casual;Action & Adventure': 0.13540961408259986, 'Photography': 2.945159106296547, 'Casual;Pretend Play': 0.23696682464454977, 'Puzzle;Education': 0.011284134506883321, 'Role Playing': 0.9365831640713158, 'Tools': 8.440532611148726, 'Art & Design;Creativity': 0.06770480704129993, 'Video Players & Editors;Music & Video': 0.022568269013766643, 'Entertainment;Brain Games': 0.07898894154818326, 'News & Magazines': 2.798465357707064, 'Food & Drink': 1.2412547957571656, 'Action': 3.1031369893929135, 'Education;Pretend Play': 0.056420672534416606, 'Arcade;Action & Adventure': 0.12412547957571654, 'Simulation;Pretend Play': 0.022568269013766643, 'Racing': 0.9930038366057323, 'Education;Brain Games': 0.033852403520649964, 'Educational;Education': 0.3949447077409162, 'Casino': 0.4287971112615662, 'Card;Action & Adventure': 0.011284134506

In [34]:
print(display_table(android_final, 1))  #will show category percentage is decending order

FAMILY : 18.934777702550214
GAME : 9.693071541412774
TOOLS : 8.451816745655607
BUSINESS : 4.5926427443015125
LIFESTYLE : 3.9043105393816293
PRODUCTIVITY : 3.8930264048747465
FINANCE : 3.7011961182577298
MEDICAL : 3.5206499661475967
SPORTS : 3.39652448657188
PERSONALIZATION : 3.3175355450236967
COMMUNICATION : 3.238546603475513
HEALTH_AND_FITNESS : 3.080568720379147
PHOTOGRAPHY : 2.945159106296547
NEWS_AND_MAGAZINES : 2.798465357707064
SOCIAL : 2.663055743624464
TRAVEL_AND_LOCAL : 2.335815842924848
SHOPPING : 2.2455427668697814
BOOKS_AND_REFERENCE : 2.143985556307831
DATING : 1.8618821936357481
VIDEO_PLAYERS : 1.7941773865944481
MAPS_AND_NAVIGATION : 1.399232678853532
FOOD_AND_DRINK : 1.2412547957571656
EDUCATION : 1.1735499887158656
ENTERTAINMENT : 0.9591514330850823
LIBRARIES_AND_DEMO : 0.9365831640713158
AUTO_AND_VEHICLES : 0.9252990295644324
HOUSE_AND_HOME : 0.8237418190024826
WEATHER : 0.8011735499887158
EVENTS : 0.7109004739336493
PARENTING : 0.6544798013992327
ART_AND_DESIGN : 0.

Google Play shows a more balanced landscape of both practical and fun apps. We can see that maximum percent of apps are from family, game and tools genre. The significant number of apps are made for practical purposes. Now, we'd like to get an idea about the kind of apps with the most users.

In [33]:
print(display_table(android_final, -4))  #frequency table of genre column

Tools : 8.440532611148726
Entertainment : 6.070864364703228
Education : 5.348679756262695
Business : 4.5926427443015125
Productivity : 3.8930264048747465
Lifestyle : 3.8930264048747465
Finance : 3.7011961182577298
Medical : 3.5206499661475967
Sports : 3.4642292936131795
Personalization : 3.3175355450236967
Communication : 3.238546603475513
Action : 3.1031369893929135
Health & Fitness : 3.080568720379147
Photography : 2.945159106296547
News & Magazines : 2.798465357707064
Social : 2.663055743624464
Travel & Local : 2.324531708417964
Shopping : 2.2455427668697814
Books & Reference : 2.143985556307831
Simulation : 2.0424283457458814
Dating : 1.8618821936357481
Arcade : 1.8505980591288649
Video Players & Editors : 1.7716091175806816
Casual : 1.7490408485669149
Maps & Navigation : 1.399232678853532
Food & Drink : 1.2412547957571656
Puzzle : 1.128413450688332
Racing : 0.9930038366057323
Role Playing : 0.9365831640713158
Libraries & Demo : 0.9365831640713158
Auto & Vehicles : 0.92529902956443

The difference between the Genres and the Category columns is not crystal clear, but one thing we can notice is that the Genres column is much more granular (it has more categories). We're only looking for the bigger picture at the moment, so we'll only work with the Category column moving forward.

### Most Popular Apps by Genre on Google Play

Next we need to find out number of installs for each app. For the Google Play market, we actually have data about the number of installs, so we should be able to get a clearer picture about genre popularity. However, the install numbers don't seem precise enough — we can see that most values are open-ended (100+, 1,000+, 5,000+, etc.):

In [26]:
display_table(android_final, 5) # the installs column

1,000,000+ : 15.741367637102236
100,000+ : 11.554953735048521
10,000,000+ : 10.516813360415256
10,000+ : 10.200857594222523
1,000+ : 8.395396073121193
100+ : 6.917174452719477
5,000,000+ : 6.838185511171294
500,000+ : 5.574362446400361
50,000+ : 4.773188896411646
5,000+ : 4.513653802753328
10+ : 3.5432182351613632
500+ : 3.2498307379823967
50,000,000+ : 2.2906793048973144
100,000,000+ : 2.1214172872940646
50+ : 1.9183028661701647
5+ : 0.7898894154818324
1+ : 0.5077860528097494
500,000,000+ : 0.2708192281651997
1,000,000,000+ : 0.22568269013766643
0+ : 0.045136538027533285
0 : 0.011284134506883321


One problem with this data is that is not precise. For instance, we don't know whether an app with 100,000+ installs has 100,000 installs, 200,000, or 350,000. However, we don't need very precise data for our purposes — we only want to get an idea which app genres attract the most users, and we don't need perfect precision with respect to the number of users.

To perform computations, however, we'll need to convert each install number to float — this means that we need to remove the commas and the plus characters, otherwise the conversion will fail and raise an error. We'll do this directly in the loop below, where we also compute the average number of installs for each genre (category).

To remove characters from strings, we can use str.replace(old, new) method. str.replace() takes in two parameters, old and new, and replaces all occurrences of old within a string with new.

In [27]:
categories_android = freq_table(android_final, 1)

for category in categories_android:
    total = 0     #sum of installs specific to a particular genre
    len_category = 0  #number of apps specific to each genre
    for app in android_final:
        category_app = app[1]
        if category_app == category:            
            n_installs = app[5]
            n_installs = n_installs.replace(',', '')
            n_installs = n_installs.replace('+', '')
            total += float(n_installs)
            len_category += 1
    avg_n_installs = total / len_category #average number of installs
    print(category, ':', avg_n_installs)

LIBRARIES_AND_DEMO : 638503.734939759
LIFESTYLE : 1437816.2687861272
ENTERTAINMENT : 11640705.88235294
HOUSE_AND_HOME : 1331540.5616438356
SHOPPING : 7036877.311557789
BOOKS_AND_REFERENCE : 8767811.894736841
MAPS_AND_NAVIGATION : 4056941.7741935486
EDUCATION : 1820673.076923077
FOOD_AND_DRINK : 1924897.7363636363
EVENTS : 253542.22222222222
FAMILY : 3694276.334922527
BEAUTY : 513151.88679245283
COMICS : 817657.2727272727
TRAVEL_AND_LOCAL : 13984077.710144928
SOCIAL : 23253652.127118643
PHOTOGRAPHY : 17805627.643678162
MEDICAL : 120616.48717948717
SPORTS : 3638640.1428571427
NEWS_AND_MAGAZINES : 9549178.467741935
DATING : 854028.8303030303
WEATHER : 5074486.197183099
ART_AND_DESIGN : 1986335.0877192982
HEALTH_AND_FITNESS : 4188821.9853479853
PERSONALIZATION : 5201482.6122448975
PRODUCTIVITY : 16787331.344927534
FINANCE : 1387692.475609756
GAME : 15560965.599534342
BUSINESS : 1712290.1474201474
AUTO_AND_VEHICLES : 647317.8170731707
TOOLS : 10682301.033377837
VIDEO_PLAYERS : 24727872.4528

On average, communication apps have the most installs: 38,456,119. This number is heavily skewed up by a few apps that have over one billion installs (WhatsApp, Facebook Messenger, Skype, Google Chrome, Gmail, and Hangouts), and a few others with over 100 and 500 million installs:

In [28]:
for app in android_final:
    if app[1] == 'COMMUNICATION' and (app[5] == '1,000,000,000+'
                                      or app[5] == '500,000,000+'
                                      or app[5] == '100,000,000+'):
        print(app[0], ':', app[5])

WhatsApp Messenger : 1,000,000,000+
imo beta free calls and text : 100,000,000+
Android Messages : 100,000,000+
Google Duo - High Quality Video Calls : 500,000,000+
Messenger – Text and Video Chat for Free : 1,000,000,000+
imo free video calls and chat : 500,000,000+
Skype - free IM & video calls : 1,000,000,000+
Who : 100,000,000+
GO SMS Pro - Messenger, Free Themes, Emoji : 100,000,000+
LINE: Free Calls & Messages : 500,000,000+
Google Chrome: Fast & Secure : 1,000,000,000+
Firefox Browser fast & private : 100,000,000+
UC Browser - Fast Download Private & Secure : 500,000,000+
Gmail : 1,000,000,000+
Hangouts : 1,000,000,000+
Messenger Lite: Free Calls & Messages : 100,000,000+
Kik : 100,000,000+
KakaoTalk: Free Calls & Text : 100,000,000+
Opera Mini - fast web browser : 100,000,000+
Opera Browser: Fast and Secure : 100,000,000+
Telegram : 100,000,000+
Truecaller: Caller ID, SMS spam blocking & Dialer : 100,000,000+
UC Browser Mini -Tiny Fast Private & Secure : 100,000,000+
Viber Mess

If we removed all the communication apps that have over 100 million installs, the average would be reduced roughly ten times:

In [29]:
under_100_m = []

for app in android_final:
    n_installs = app[5]
    n_installs = n_installs.replace(',', '')
    n_installs = n_installs.replace('+', '')
    if (app[1] == 'COMMUNICATION') and (float(n_installs) < 100000000):
        under_100_m.append(float(n_installs))
        
sum(under_100_m) / len(under_100_m)

3603485.3884615386

We see the same pattern for the video players category, which is the runner-up with 24,727,872 installs. The market is dominated by apps like Youtube, Google Play Movies & TV, or MX Player. The pattern is repeated for social apps (where we have giants like Facebook, Instagram, Google+, etc.), photography apps (Google Photos and other popular photo editors), or productivity apps (Microsoft Word, Dropbox, Google Calendar, Evernote, etc.).

Again, the main concern is that these app genres might seem more popular than they really are. Moreover, these niches seem to be dominated by a few giants who are hard to compete against.

The game genre seems pretty popular, but previously we found out this part of the market seems a bit saturated, so we'd like to come up with a different app recommendation if possible.

The books and reference genre looks fairly popular as well, with an average number of installs of 8,767,811. It's interesting to explore this in more depth, since we found this genre has some potential to work well on the App Store, and our aim is to recommend an app genre that shows potential for being profitable on Google Play.

Let's take a look at some of the apps from this genre and their number of installs:

In [35]:
for app in android_final:
    if app[1] == 'BOOKS_AND_REFERENCE':
        print(app[0], ':', app[5])

E-Book Read - Read Book for free : 50,000+
Download free book with green book : 100,000+
Wikipedia : 10,000,000+
Cool Reader : 10,000,000+
Free Panda Radio Music : 100,000+
Book store : 1,000,000+
FBReader: Favorite Book Reader : 10,000,000+
English Grammar Complete Handbook : 500,000+
Free Books - Spirit Fanfiction and Stories : 1,000,000+
Google Play Books : 1,000,000,000+
AlReader -any text book reader : 5,000,000+
Offline English Dictionary : 100,000+
Offline: English to Tagalog Dictionary : 500,000+
FamilySearch Tree : 1,000,000+
Cloud of Books : 1,000,000+
Recipes of Prophetic Medicine for free : 500,000+
ReadEra – free ebook reader : 1,000,000+
Anonymous caller detection : 10,000+
Ebook Reader : 5,000,000+
Litnet - E-books : 100,000+
Read books online : 5,000,000+
English to Urdu Dictionary : 500,000+
eBoox: book reader fb2 epub zip : 1,000,000+
English Persian Dictionary : 500,000+
Flybook : 500,000+
All Maths Formulas : 1,000,000+
Ancestry : 5,000,000+
HTC Help : 10,000,000+
E

In [31]:
for app in android_final:
    if app[1] == 'BOOKS_AND_REFERENCE' and (app[5] == '1,000,000,000+'
                                            or app[5] == '500,000,000+'
                                            or app[5] == '100,000,000+'):
        print(app[0], ':', app[5])


Google Play Books : 1,000,000,000+
Bible : 100,000,000+
Amazon Kindle : 100,000,000+
Wattpad 📖 Free Books : 100,000,000+
Audiobooks from Audible : 100,000,000+


In [32]:
for app in android_final:
    if app[1] == 'BOOKS_AND_REFERENCE' and (app[5] == '1,000,000+'
                                            or app[5] == '5,000,000+'
                                            or app[5] == '10,000,000+'
                                            or app[5] == '50,000,000+'):
        print(app[0], ':', app[5])

Wikipedia : 10,000,000+
Cool Reader : 10,000,000+
Book store : 1,000,000+
FBReader: Favorite Book Reader : 10,000,000+
Free Books - Spirit Fanfiction and Stories : 1,000,000+
AlReader -any text book reader : 5,000,000+
FamilySearch Tree : 1,000,000+
Cloud of Books : 1,000,000+
ReadEra – free ebook reader : 1,000,000+
Ebook Reader : 5,000,000+
Read books online : 5,000,000+
eBoox: book reader fb2 epub zip : 1,000,000+
All Maths Formulas : 1,000,000+
Ancestry : 5,000,000+
HTC Help : 10,000,000+
Moon+ Reader : 10,000,000+
English-Myanmar Dictionary : 1,000,000+
Golden Dictionary (EN-AR) : 1,000,000+
All Language Translator Free : 1,000,000+
Aldiko Book Reader : 10,000,000+
Dictionary - WordWeb : 5,000,000+
50000 Free eBooks & Free AudioBooks : 5,000,000+
Al-Quran (Free) : 10,000,000+
Al Quran Indonesia : 10,000,000+
Al'Quran Bahasa Indonesia : 10,000,000+
Al Quran Al karim : 1,000,000+
Al Quran : EAlim - Translations & MP3 Offline : 5,000,000+
Koran Read &MP3 30 Juz Offline : 1,000,000+
H

This niche seems to be dominated by software for processing and reading ebooks, as well as various collections of libraries and dictionaries, so it's probably not a good idea to build similar apps since there'll be some significant competition.

We also notice there are quite a few apps built around the book Quran, which suggests that building an app around a popular book can be profitable. It seems that taking a popular book (perhaps a more recent book) and turning it into an app could be profitable for both the Google Play market.

However, it looks like the market is already full of libraries, so we need to add some special features besides the raw version of the book. This might include daily quotes from the book, an audio version of the book, quizzes on the book, a forum where people can discuss the book, etc.

### CONCLUSION
In this project, we analyzed data about the App Store and Google Play mobile apps with the goal of recommending an app profile that can be profitable for both markets.

We concluded that taking a popular book (perhaps a more recent book) and turning it into an app could be profitable for both the Google Play and the App Store markets. The markets are already full of libraries, so we need to add some special features besides the raw version of the book. This might include daily quotes from the book, an audio version of the book, quizzes on the book, a forum where people can discuss the book, etc.